In [4]:
!pip install pyspark
!pip install pandas
import pandas as pd
global_train_df = pd.read_csv("../MATHM0029_2024_TB-1/global_train_df.csv")
global_test_df = pd.read_csv("../MATHM0029_2024_TB-1/global_test_df.csv")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great

In [15]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder

# Encode user and item metadata
user_metadata_columns = ['Age', 'Gender', 'Occupation']
item_metadata_columns = list(global_train_df.columns[9:])  # All columns from 'unknown' to 'Western'

# Label encode categorical columns like Gender and Occupation
label_encoders = {col: LabelEncoder() for col in user_metadata_columns if global_train_df[col].dtype == 'object'}

for col, encoder in label_encoders.items():
    global_train_df[col] = encoder.fit_transform(global_train_df[col])
    global_test_df[col] = encoder.transform(global_test_df[col])

# Aggregate genre metadata for items
# Ensure all genre columns are numeric and handle non-numeric data
for col in item_metadata_columns:
    global_train_df[col] = pd.to_numeric(global_train_df[col], errors='coerce')

# Re-aggregate item genres after converting to numeric
item_genres = global_train_df.groupby('Item ID')[item_metadata_columns].mean()

# Verify the aggregated data to ensure no issues
item_genres.head()

# Merge user metadata with ratings in train/test sets
# Rename user metadata columns to avoid overlap
user_metadata = user_metadata.rename(columns={
    'Age': 'User_Age',
    'Gender': 'User_Gender',
    'Occupation': 'User_Occupation'
})

global_train_df = global_train_df.set_index('User ID').join(user_metadata).reset_index()
global_test_df = global_test_df.set_index('User ID').join(user_metadata).reset_index()

# Verify the preprocessed data structure
global_train_df.head(), global_test_df.head(), item_genres.head()



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


(   User ID  Item ID  Rating  timestamp  Age  Gender  Occupation zip code  \
 0      864     1044       3  888891049   27       1          14    63021   
 1      864      159       4  888891049   27       1          14    63021   
 2      864     1303       2  888890997   27       1          14    63021   
 3      864      184       4  888890775   27       1          14    63021   
 4      864     1531       3  888890690   27       1          14    63021   
 
                                          Movie Title  Release Date  ...  \
 0                                  Paper, The (1994)           NaN  ...   
 1                              Basic Instinct (1992)           NaN  ...   
 2                                Getaway, The (1994)           NaN  ...   
 3                            Army of Darkness (1993)           NaN  ...   
 4  Far From Home: The Adventures of Yellow Dog (1...           NaN  ...   
 
    Musical  Mystery  Romance  Sci-Fi  Thriller  War  Western  User_Age  \
 0 

In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("MatrixFactorization").getOrCreate()

# Convert training and test datasets to Spark DataFrames
global_train_sdf = spark.createDataFrame(global_train_df[['User ID', 'Item ID', 'Rating']])
global_test_sdf = spark.createDataFrame(global_test_df[['User ID', 'Item ID', 'Rating']])

# Define ALS model
als = ALS(
    userCol="User ID",
    itemCol="Item ID",
    ratingCol="Rating",
    maxIter=10,
    regParam=0.1,
    rank=10,
    coldStartStrategy="drop"
)

# Train the ALS model
als_model = als.fit(global_train_sdf)

# Evaluate the model on test data
predictions = als_model.transform(global_test_sdf)

# Calculate RMSE for evaluation
evaluator_rmse = predictions.select((col("Rating") - col("prediction")) ** 2).agg({"POWER((Rating - prediction), 2)": "mean"})
rmse = evaluator_rmse.first()[0] ** 0.5

rmse


1.0462375221696674

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Combine ALS embeddings and metadata
user_combined_features = user_embeddings.drop('features', axis=1).join(pd.DataFrame(user_embeddings['features'].to_list()))
item_combined_features = item_embeddings.drop('features', axis=1).join(pd.DataFrame(item_embeddings['features'].to_list()))

# Compute cosine similarity between user and item combined features
user_vectors = np.array(user_combined_features.iloc[:, 1:])
item_vectors = np.array(item_combined_features.iloc[:, 1:])
similarity_matrix = cosine_similarity(user_vectors, item_vectors)

# Generate top-k recommendations for each user
top_k_recommendations = np.argsort(-similarity_matrix, axis=1)[:, :10]


NameError: name 'user_embeddings' is not defined